Maxwell solver for Pec using direct formulation
=============================
**keys**: Maxwell double layer potential, PEC scattering, MoM, Neumann trace

In [ ]:
from netgen.occ import *
import netgen.meshing as meshing
from ngsolve import *
from ngsolve.webgui import Draw
from libbem import *
from ngsolve import Projector, Preconditioner
from ngsolve.krylovspace import CG

In [ ]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=1, perfstepsend=meshing.MeshingStep.MESHSURFACE)).Curve(3)

In [ ]:
fesHCurl = HCurl(mesh, order=3, complex=True)
fesHDiv = HDivSurface(mesh, order=3, complex=True)
uHCurl,vHCurl = fesHCurl.TnT() # H(curl_Gamma) trial for Dirichlet data ( (nxE)xn )
uHDiv,vHDiv = fesHDiv.TnT() # H(div_Gamma) trial space for Neumann data ( nx curlE ) and test space for BIE

print ("ndof HCurl = ", fesHCurl.ndof)
print ("ndof HDiv = ", fesHDiv.ndof)

In [ ]:
eps0 = 8.854e-12 
mu0 = 4*pi*1e-7
omega = 1.5e9
kappa = omega*sqrt(eps0*mu0)
print("kappa = ", kappa)

E_inc = CF((1,0,0))*exp( -1j * kappa * z )

n = specialcf.normal(3)
mR = GridFunction(fesHCurl) 
mR.Set( - Cross( Cross(n, E_inc), n) , definedon=mesh.Boundaries(".*"), dual=True) # Hcurl

In [ ]:
# new operators, i.e., V: Hdiv is TnT; K: Hdiv as test and HCurl as trial space with rotated evaluator   
with TaskManager(): 
    V = MaxwellSingleLayerPotentialOperator(fesHDiv, kappa, 
                                            intorder=10, leafsize=40, eta=3., eps=1e-4, method="aca", testhmatrix=False)
    K = MaxwellDoubleLayerPotentialOperator(fesHCurl, fesHDiv, kappa, 
                                            intorder=12, leafsize=40, eta=3., eps=1e-6, method="aca", testhmatrix=False)

In [ ]:
# solve direct formulation for j, i.e., Neumann data of scattered electric field E
j = GridFunction(fesHDiv) 
pre = BilinearForm( uHDiv.Trace() * vHDiv.Trace() *ds).Assemble().mat.Inverse(freedofs=fesHDiv.FreeDofs()) 
with TaskManager(): 
    M = BilinearForm( uHCurl.Trace() * vHDiv.Trace()* ds(bonus_intorder=3)).Assemble() # <Hcurl, Hdiv>  
    rhs = ( (0.5 * M.mat - K.mat ) * mR.vec).Evaluate() 
    CG(mat = V.mat, pre=pre, rhs = rhs, sol=j.vec, tol=1e-8, maxsteps=500, initialize=False, printrates=False)

# sove EFIE, j_efie
j_efie = GridFunction(fesHDiv) # Hdiv
rhs_efie = LinearForm( mR * vHDiv.Trace() *ds(bonus_intorder=3)).Assemble() 
CG(mat = V.mat, pre=pre, rhs = rhs_efie.vec, sol=j_efie.vec, tol=1e-8, maxsteps=500, initialize=False, printrates=False);
curlE_inc = CF( (0,-1j*kappa,0) ) *exp( -1j * kappa * z ) 

In [ ]:
# check j: it must hold j_efie =j + j_inc 
# reason: j_efie is the Neumann data of total electric Field E_tot = E + E_inc
j_inc = GridFunction(fesHDiv) 
j_inc.Set( Cross(n, curlE_inc), definedon=mesh.Boundaries(".*"), dual=True) # Hdiv 

j_efie_test = GridFunction(fesHDiv)
j_efie_test.Set (j + j_inc, definedon=mesh.Boundaries(".*"), dual=True) # consistent in Hdiv

Draw(Norm(j_efie), mesh, draw_vol=False, order=3);
Draw(Norm(j_efie_test), mesh, draw_vol=False, order=3);